- 使用dataset(roberthsu2003/data_for_RAG)
- 使用模型(使用Microsoft開源的intfloat/multilingual-e5-large)
- 模型大約要2.24GB

In [2]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from datasets import load_dataset

class DocumentSearch:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
        self.model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)

    def get_embedding(self, text, is_query=False):
        #增加前綴文字
        if is_query:
            text = f"query:{text}"
        else:
            text = f"passage:{text}"
        
        #編碼文本
        inputs = self.tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        

        #獲取嵌入向量
        with torch.no_grad():
            outputs = self.model(**inputs)
            #如果依據測試檢索能力的寫法是
            #embeddings = outputs.last_hidden_state.mean(dim=1)
            #但這種方法會比較好,看下面的說明1,2
            embeddings = outputs.last_hidden_state[:, 0, :] #使用[CLS]token的輸出#[CLS]標記通常被用來表示整個句子的語義
            embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
        return embeddings.cpu().numpy()[0] #請看說明3

    def create_document_embeddings(self, df):
        """為輸入的dataFrame建立嵌入向量"""
        embeddings = []
        for _, row in df.iterrows():
            #組合Title和Text欄位
            text = f"Title:{row['Title']}\nText:{row['Text']}"
            embedding = self.get_embedding(text, is_query=False)
            embeddings.append(embedding)
        return embeddings
    
    def find_best_passage(self, query, df):
        """查載最相關的文本"""
        query_embedding = self.get_embedding(query, is_query=True)

        #計算相似度
        similarities = np.dot(np.stack(df['Embeddings'].values), query_embedding)#請看說明4

        #傳回最相關的文檔
        best_idx = np.argmax(similarities)
        return df.iloc[best_idx]['Text'], similarities[best_idx]

In [8]:
doc_search = DocumentSearch()

# 準備數據
datasets = load_dataset('roberthsu2003/data_for_RAG')
df = pd.DataFrame(datasets['train'])
df.columns = ['Title','Text']
# 生成嵌入向量
df['Embeddings'] = doc_search.create_document_embeddings(df)



In [11]:
# 測試查詢
query = "有多少估USB連接埠"

best_text, similarity = doc_search.find_best_passage(query, df)
print(f"Query:{query}")
print(f"最相關的文字是(相似度:{similarity:.3f}):")
print(best_text)

Query:有多少估USB連接埠
最相關的文字是(相似度:0.893):
Model 3 共有四個 USB 連接埠。一個 USB-C 在中控台後方置物空間，兩個 USB-C 在中控台後車廂出風口下方，一個 USB-A 在手套箱內。[source: 334]
